In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import cPickle as pickle
import numpy as np
import os
with open('./cifar10/cifar-10-batches-py/data_batch_1', 'rb') as f:
    datadict = pickle.load(f)
    X = datadict['data']

In [3]:
from cifar import load_CIFAR10
plt.rcParams['figure.figsize'] = (10.0, 8.0) 

cifar10_dir = './cifar10/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)
samples_per_class = 7
for y, cls in enumerate(classes):
    idxs = np.flatnonzero(y_train == y)
    idxs = np.random.choice(idxs, samples_per_class, replace=False)
    for i, idx in enumerate(idxs):
        plt_idx = i * num_classes + y + 1
        plt.subplot(samples_per_class, num_classes, plt_idx)
        plt.imshow(X_train[idx].astype('uint8').reshape(32, 32, 3))
        plt.axis('off')
        if i == 0:
            plt.title(cls)
plt.show()

In [4]:
X_train = X_train.reshape(-1,3,32,32)
X_test = X_test.reshape(-1,3,32,32)

In [5]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *

input_X = T.tensor4("X", dtype = 'float32')
target_y = T.vector("target Y integer",dtype='int64')


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release.  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 89.0% of memory, cuDNN 5105)


Так задаётся архитектура нейронки

In [6]:
from lasagne.layers import Conv2DLayer
from lasagne.layers import MaxPool2DLayer
input_layer = lasagne.layers.InputLayer(shape=(None,3, 32, 32), input_var=input_X)
#nnet = <сверочная нейросеть>
# Для вдохновления 
# - http://torch.ch/blog/2015/07/30/cifar.html
# - http://www.robots.ox.ac.uk/~vgg/research/very_deep/
# - https://www.cs.unc.edu/~wliu/papers/GoogLeNet.pdf
nnet = lasagne.layers.Conv2DLayer(    
            input_layer, num_filters=64,
            filter_size=(5, 5), pad='same',
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(gain='relu')
            )
print(nnet.output_shape)

nnet = lasagne.layers.Conv2DLayer(
            nnet, num_filters=64,
            filter_size=(5, 5),pad='same',
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(gain='relu')
            )
print(nnet.output_shape)

nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
print(nnet.output_shape)

nnet = lasagne.layers.Conv2DLayer(
            nnet, num_filters=128,
            filter_size=(5, 5),pad='same',
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(gain='relu')
            )
nnet = lasagne.layers.Conv2DLayer(
            nnet, num_filters=128,
            filter_size=(5, 5),pad='same',
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(gain='relu')
            )
print(nnet.output_shape)
nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
print(nnet.output_shape)

nnet = lasagne.layers.Conv2DLayer(
            nnet, num_filters=256,
            filter_size=(5, 5),pad='same',
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(gain='relu')
            )
nnet = lasagne.layers.Conv2DLayer(
            nnet, num_filters=256,
            filter_size=(5, 5),pad='same',
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(gain='relu')
            )
print(nnet.output_shape)
nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
print(nnet.output_shape)

nnet = lasagne.layers.Conv2DLayer(
            nnet, num_filters=512,
            filter_size=(3, 3),pad='same',
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(gain='relu')
            )
nnet = lasagne.layers.Conv2DLayer(
            nnet, num_filters=512,
            filter_size=(3, 3),pad='same',
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(gain='relu')
            )
print(nnet.output_shape)
nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
print(nnet.output_shape)

nnet = lasagne.layers.Conv2DLayer(
            nnet, num_filters=1024,
            filter_size=(1, 1),pad='same',
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(gain='relu')
            )
nnet = lasagne.layers.Conv2DLayer(
            nnet, num_filters=1024,
            filter_size=(1, 1),pad='same',
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(gain='relu')
            )
print(nnet.output_shape)
nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
print(nnet.output_shape)

dense_output = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(nnet, p=.5),
        num_units=10,
        nonlinearity=lasagne.nonlinearities.softmax)




 

(None, 64, 32, 32)
(None, 64, 32, 32)
(None, 64, 16, 16)
(None, 128, 16, 16)
(None, 128, 8, 8)
(None, 256, 8, 8)
(None, 256, 4, 4)
(None, 512, 4, 4)
(None, 512, 2, 2)
(None, 1024, 2, 2)
(None, 1024, 1, 1)


In [7]:
#предсказание нейронки (theano-преобразование)
y_predicted = lasagne.layers.get_output(dense_output)

In [8]:
#все веса нейронки (shared-переменные)
all_weights = lasagne.layers.get_all_params(dense_output, trainable=True)

### дальше вы могли бы просто
* задать функцию ошибки вручную
* посчитать градиент ошибки по all_weights
* написать updates
* но это долго, а простой шаг по градиенту - не самый лучший смособ оптимизировать веса

Вместо этого, опять используем lasagne

In [9]:
loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean()
accuracy = lasagne.objectives.categorical_accuracy(y_predicted,target_y).mean()
updates = lasagne.updates.adam(loss, all_weights)

In [10]:
train_fun = theano.function([input_X,target_y],[loss, accuracy], updates= updates, allow_input_downcast=True )
accuracy_fun = theano.function([input_X,target_y],accuracy, allow_input_downcast=True)

### Вот и всё, пошли её учить

In [11]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

# Процесс обучения

In [13]:
import time

num_epochs = 58 #количество проходов по данным

batch_size = 150 #размер мини-батча

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Epoch 1 of 58 took 283.871s
  training loss (in-iteration):		1.634891
  train accuracy:		39.62 %
  validation accuracy:		42.84 %
Epoch 2 of 58 took 282.479s
  training loss (in-iteration):		1.541642
  train accuracy:		43.94 %
  validation accuracy:		43.03 %
Epoch 3 of 58 took 283.167s
  training loss (in-iteration):		1.479293
  train accuracy:		46.24 %
  validation accuracy:		45.10 %
Epoch 4 of 58 took 282.548s
  training loss (in-iteration):		1.433143
  train accuracy:		48.31 %
  validation accuracy:		45.86 %
Epoch 5 of 58 took 281.488s
  training loss (in-iteration):		1.382827
  train accuracy:		50.24 %
  validation accuracy:		47.47 %
Epoch 6 of 58 took 281.291s
  training loss (in-iteration):		1.351014
  train accuracy:		51.60 %
  validation accuracy:		47.20 %
Epoch 7 of 58 took 281.258s
  training loss (in-iteration):		1.317643
  train accuracy:		52.80 %
  validation accuracy:		47.22 %
Epoch 8 of 58 took 281.203s
  training loss (in-iteration):		1.295277
  train accuracy:		53.47 %


In [13]:
test_acc = 0
test_batches = 2
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 80:
    print "Achievement unlocked: колдун 80 уровня"
else:
    print "Нужно больше магии!"

Final results:
  test accuracy:		30.38 %
Нужно больше магии!


## Заполните форму

https://goo.gl/forms/FsANPB1jSqmX1JBJ3